# Boxing Bag Hit Detection and Count
The following code involved object detection to detect hits on a punching bag during a training session.

This notebook starts by setting up the environment, including checking GPU availability with the nvidia-smi command to ensure that the model can utilize GPU acceleration for faster performance. Key dependencies, such as ultralytics for the YOLOv8 model and roboflow for dataset management, are installed to streamline the workflow.

The next step involves integrating the Roboflow API to download the boxing glove detection dataset. This dataset, annotated for detecting boxing gloves, is retrieved in YOLOv8 format, making it directly compatible with the object detection model used in the project. The notebook then proceeds to initialize and train the YOLOv8 model using the downloaded dataset. The model is trained over 200 epochs, with an image size of 640 pixels, to enhance its ability to detect boxing gloves in frames accurately.

Once the model is trained, it is utilized for real-time boxing glove detection. Using OpenCV, the notebook captures frames from a live video feed and passes them to the YOLOv8 model for detection. The model outputs bounding boxes around detected gloves, which are displayed on the live video feed, allowing for visual feedback of glove detection.

The notebook also includes a hit detection and counting feature. A custom function is implemented to count the number of hits by detecting collisions between the boxing gloves and a target (e.g., a boxing bag). This collision detection method compares the bounding boxes of gloves and the target to identify overlaps, which signify a hit. A cooldown mechanism is introduced to avoid multiple hits being counted in consecutive frames, ensuring accurate counting. The model is tested on pre-recorded videos, and the total number of detected hits is displayed at the end of the session.

In [ ]:
!nvidia-smi

Sun Sep 15 09:41:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


### We will be using YOLO8 model to perform this:

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 32.6/235.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8


In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
import cv2
import numpy as np
from roboflow import Roboflow
rf = Roboflow(api_key="API KEY") #NOTE: You can get your own API key by going to the dataset on roboflow through the link in the documentation and clicking download project>yolov8>show download code
project = rf.workspace("ie").project("boxing-club")
version = project.version(2)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Boxing-club-2 in yolov8:: 100%|██████████| 1724/1724 [00:00<00:00, 4153.66it/s]


In [ ]:
model = YOLO("yolov8n.yaml")
model.train(data="/content/Boxing-club-2/data.yaml", epochs=200, imgsz=640)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x793a02da3520>
fitness: 0.5963064742690027
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.69615,     0.47099,     0.60817,     0.44165,     0.53398,     0.69356])
names: {0: 'bag', 1: 'cross', 2: 'hook', 3: 'jab', 4: 'no punch', 5: 'uppercut'}
plot: True
results_dict: {'metrics/precision(B)': 0.8676649217465436, 'metrics/recall(B)': 0.6998710718149783, 'metrics/mAP50(B)': 0.7963280478166261, 'metrics/mAP50-95(B)': 0.5740818549859336, 'fitness': 0.5963064742690027}
save_dir: PosixPath('runs/detect/train')
speed: {'preprocess': 0.1994197288256013, 'inference': 2.069786693272966, 'loss': 0.0015671333570158885, 'postprocess': 11.910226907622949}

## Let us try and predict the number of hits on a video.
We use the following videos to test this from this google drive: [Link](https://drive.google.com/drive/folders/1UwZPZ7sqkmQrqbCP1ypquv2UHWkk0bj-)
The above google drive is from the following punch classification project on github [link ](https://github.com/balezz/Punch_DL?tab=readme-ov-file)

In [ ]:
results = model.predict(source="/content/id0_hook_1.mp4", save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 57.8ms
video 1/1 (2/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 7.1ms
video 1/1 (3/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 7.0ms
video 1/1 (4/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 8.0ms
video 1/1 (5/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 8.4ms
video 1/1 (6/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 8.1ms
video 1/1 (7/1100) /content/id0_hook_1.mp4: 640x384 1 bag, 7.4ms


Let us count each type of predicted move

In [ ]:
def count_moves(predictions):
    # Initialize counters for each type of move
    cross_count = 0
    hook_count = 0
    jab_count = 0
    uppercut_count = 0
    for pred in predictions:

        boxes = pred.boxes
        for box in boxes:
            class_label = box.cls
            if class_label == 1:  # Class '1' is 'cross'
                cross_count += 1
            elif class_label == 2:  # Class '2' is 'hook'
                hook_count += 1
            elif class_label == 3:  # Class '3' is 'jab'
                jab_count += 1
            elif class_label == 5:  # Class '5' is 'uppercut'
                uppercut_count += 1

    print(f"Total Crosses Detected: {cross_count}")
    print(f"Total Hooks Detected: {hook_count}")
    print(f"Total Jabs Detected: {jab_count}")
    print(f"Total Uppercuts Detected: {uppercut_count}")

    return cross_count, hook_count, jab_count, uppercut_count
cross_count, hook_count, jab_count, uppercut_count = count_moves(results)


Total Crosses Detected: 1
Total Hooks Detected: 18
Total Jabs Detected: 24
Total Uppercuts Detected: 15


As we can see, the above model is not accurate as the video we analyzed only contains 20 punches. Hence, let us try to do the same using collision detection

# Hit detection using collision detection

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="APIKEY")#NOTE: You can get your own API key by going to the dataset on roboflow through the link in the documentation and clicking download project>yolov8>show download code
project = rf.workspace("computer-vision-jrnie").project("boxing-detected")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Boxing-Detected-2 in yolov8:: 100%|██████████| 262/262 [00:00<00:00, 6398.11it/s]


In [ ]:
model1 = YOLO("yolov8n.yaml")
model1.train(data="/content/Boxing-Detected-2/data.yaml", epochs=300, imgsz=640,batch=32,lr0=0.001,augment=True)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x793744703af0>
fitness: 0.450448436326581
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.54674,     0.49163,     0.21944])
names: {0: 'Boxing Bag', 1: 'Boxing Gloves', 2: 'Boxing'}
plot: True
results_dict: {'metrics/precision(B)': 0.7233280721792233, 'metrics/recall(B)': 0.6499809483172793, 'metrics/mAP50(B)': 0.7310349299767189, 'metrics/mAP50-95(B)': 0.41927215925434347, 'fitness': 0.450448436326581}
save_dir: PosixPath('runs/detect/train2')
speed: {'preprocess': 0.19940308162144252, 'inference': 12.950505529131208, 'loss': 0.0009196145193917411, 'postprocess': 0.7936443601335798}

In [ ]:
results = model1.predict(source="/content/id0_hook_1.mp4", save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1100) /content/id0_hook_1.mp4: 640x384 2 Boxing Bags, 1 Boxing Gloves, 2 Boxings, 80.7ms
video 1/1 (2/1100) /content/id0_hook_1.mp4: 640x384 1 Boxing Bag, 1 Boxing Gloves, 2 Boxings, 24.5ms
video 1/1 (3/1100) /content/id0_hook_1.mp4: 640x384 1 Boxing Bag, 1 Boxing Gloves, 5 Boxings, 25.2ms
video 1/1 (4/1100) /content/id0_hook_1.mp4: 640x384 1 Boxing Bag, 1 Boxing Gloves, 2 Boxings, 25.1ms
video 1/1 (5/1100) /content/id0_hook_1.mp4: 640x384

Let us write a colission detection script. The function checks if two bounding boxes overlap by comparing their minimum and maximum x and y coordinates. If there is an overlap, it returns true that indicates collision. The count_hits function then iterates through the predictions to check for collisions between the boxing bag and the boxing gloves class and counts the number of hits. There is also a cooldown mechanism such that multiple hits are avoided in consecutive frames by setting a cooldown period.


In [ ]:
import cv2

def detect_collision(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    if x1_max < x2_min or x1_min > x2_max:
        return False
    if y1_max < y2_min or y1_min > y2_max:
        return False
    return True

def count_hits(predictions, cooldown_frames=50):
    hits = 0
    cooldown = 0

    for pred in predictions:
        if cooldown > 0:
            cooldown -= 1
            continue
        boxes = pred.boxes
        boxing_boxes = [box.xyxy[0].tolist() for box in boxes if box.cls == 2]  # Assuming class 0 is "Boxing"
        gloves_boxes = [box.xyxy[0].tolist() for box in boxes if box.cls == 1]  # Assuming class 1 is "Boxing Gloves"

        for boxer_box in boxing_boxes:
            for glove_box in gloves_boxes:
                if detect_collision(boxer_box, glove_box):
                    hits += 1
                    cooldown = cooldown_frames
                    break

    return hits
hits = count_hits(results, cooldown_frames=50)

print(f"Total Hits Detected: {hits}")




Total Hits Detected: 21


In [ ]:
results1 = model1.predict(source="/content/id0_jab_1.mp4", save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/952) /content/id0_jab_1.mp4: 640x384 2 Boxing Bags, 1 Boxing Gloves, 3 Boxings, 33.0ms
video 1/1 (2/952) /content/id0_jab_1.mp4: 640x384 2 Boxing Bags, 1 Boxing Gloves, 5 Boxings, 33.0ms
video 1/1 (3/952) /content/id0_jab_1.mp4: 640x384 2 Boxing Bags, 1 Boxing Gloves, 1 Boxing, 28.6ms
video 1/1 (4/952) /content/id0_jab_1.mp4: 640x384 1 Boxing Bag, 1 Boxing Gloves, 1 Boxing, 28.3ms
video 1/1 (5/952) /content/id0_jab_1.mp4: 640x384 1 Boxing 

In [ ]:
hits = count_hits(results1, cooldown_frames=45)
print(f"Total Hits Detected: {hits}")

Total Hits Detected: 21


As we can see, this accurately detected the number of punches in both videos